In [ ]:
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import datetime as dt


In [ ]:
df = pd.read_csv("nyc_carpooling.csv")

In [ ]:
dff = df[((df.PULocationID >= 200) & (df.PULocationID < 220)) & ((df.DOLocationID > 200) & (df.DOLocationID <= 220)) & (df.PULocationID != df.DOLocationID) & df.trip_distance > 0.0]
dff

In [ ]:
sff = dff.groupby(['PULocationID', 'DOLocationID']).min().reset_index()
sff

In [ ]:
df1=df[["PULocationID","DOLocationID","trip_distance"]]
df1.head()

In [ ]:
def buildgraph(cab_dataset):
    unique_points_pickup = set(cab_dataset['PULocationID'])
    unique_points_drop = set(cab_dataset['DOLocationID'])
    unique_points = unique_points_pickup.union(unique_points_drop)

    matrix = [[0]*(len(unique_points)) for i in range(len(unique_points))]  
    pickup = list(cab_dataset['PULocationID'])
    drop = list(cab_dataset['DOLocationID'])
    dist = list(cab_dataset['trip_distance'])

    dictionary = {}
    dictionary2 = {}
    for index, val in enumerate(unique_points):
        dictionary[val] = index
    
    for i in range(len(pickup)):
        dictionary2[(pickup[i],drop[i])] = dist[i]
 
    for p in pickup:
        for d in drop:
            i = dictionary[p]
            j = dictionary[d]
            try:
                val = dictionary2[(p,d)]
                matrix[i][j] = val
            except:
                continue
    return matrix

In [ ]:
matrix = (buildgraph(df1))

In [ ]:
min_cost = 1000000000
busRoute = []
source = 200
thresholds = dict({
        "seats": 8,
        "max_buses": 5,
        "max_denials": 5
    })

In [ ]:
pass_data = pd.read_csv("Passenger_Data.csv")
pass_data

In [ ]:
filtered_data = pass_data[(pass_data.booking_timestamp >= 100000) & (pass_data.booking_timestamp <= 110000)]
filtered_data

In [ ]:
def get_dropoff_by_passId(passenger_ids):
    dropoff_loc_ids = []
    for passenger in passenger_ids:
        dropoff_loc_ids.append(int(filtered_data[filtered_data.passenger_id == passenger].dropoff_location))
    return dropoff_loc_ids

In [ ]:
def findMinCostRoute(graph, visited, req_nodes, src, dest, current_cost):
    global min_cost
    global busRoute

    # If destination is reached
    if (src == dest):
        # Set flag
        flag = True
        # Visit all the intermediate nodes
        for i in req_nodes:
            # If any intermediate node is not visited, clear flag
            if not visited[i]:
                flag = False
                break

        # If all intermediate nodes are visited, return
        if (flag):
            # Update the min_cost
            min_cost = min(min_cost, current_cost)
            busRoute.append(src)
        return;

    else:
        # set the current node as visited
        visited[src] = True
        busRoute.append(src)
        # Traverse adjacent locations
        for node in graph[src]:
            if not visited[node[0]]:
                # Mark the adjacent node visited
                visited[node[0]] = True
                # Find minimum cost path with adjacent node as source
                findMinCostRoute(graph, visited, req_nodes, node[0], dest, current_cost + node[1]);
                # Clear visited adjacent node
                visited[node[0]] = False

        # Mark the source unvisited
        visited[src] = False;

In [ ]:
def allocatePassengers(dropoff_locations, source):
    # unique_nodes = dropoff_locations
    unique_nodes = set(dropoff_locations)
    # unique_nodes = {x for x in dropoff_locations}
    graph = prepare_graph(unique_nodes, source)
    return invokeDFS_Algo(graph, unique_nodes, source)

In [ ]:
def prepare_graph(unique_nodes, source):
    graph = dict()
    for dropId in unique_nodes:
        #print("drop, ", dropId, " ", matrix[dropId - 200][:218-200])
        tmp_matrix_row = matrix[dropId - source]
        edges = []
        for id in unique_nodes:
            if id == dropId:
                continue
            value = 99 if tmp_matrix_row[id - source] == 0 else tmp_matrix_row[id - source]
            edges.append([id, value])
        graph[dropId] = edges

    # append source
    source_row = matrix[0]
    edges = []
    for id in unique_nodes:
        value = 99 if source_row[id - source] == 0 else source_row[id - source]
        edges.append([id, value])
    graph[source] = edges

    return graph

In [ ]:
from collections import OrderedDict

def invokeDFS_Algo(graph, unique_nodes, source):
    # Number of nodes
    n = len(unique_nodes);
    min_cost = 99999999999
    best_route = []
    for destination in unique_nodes:
        # Destination
        dest = destination;

        vis = dict()
        for i in unique_nodes:
            vis[i] = False

        # Stores intermediate nodes
        necessary = [];
        for i in unique_nodes:
            if i != source and i != dest:
                necessary.append(i)
        busRoute = []
        findMinCostRoute(graph, vis, necessary, source, dest, 0);
        # If no path is found
        if min_cost == 1000000000:
            print(-1)
        else:
            #print(min_cost)
            #print("route", list(OrderedDict.fromkeys(busRoute)))
            if min_cost < min_cost:
                print("busRoute", busRoute)
                best_route = list(OrderedDict.fromkeys(busRoute))
                min_cost = min_cost

        # print("best_route first", best_route)
        return min_cost, best_route


In [ ]:
from itertools import combinations

def process_passenger_batch(passenger_batch):

    higher_priority_pass =  passenger_batch[passenger_batch.denials >= 5]
    highest_priority_passengers = list(higher_priority_pass.passenger_id)
    remaining_seats = 15 - len(higher_priority_pass)

    data = passenger_batch[passenger_batch.denials < 5].head(remaining_seats)
    passenger_list = list(data.passenger_id)
    comb = combinations(passenger_list, 8 - len(higher_priority_pass))
    min_cost_of_all = 9999999
    best_route_of_all = []
    final_input_of_all = []
    for input in comb:
        final_input = list(input) + highest_priority_passengers
        unique_nodes = get_dropoff_by_passId(final_input)
        cost, route = allocatePassengers(unique_nodes, source)
        if cost < min_cost_of_all:
            best_route_of_all = route
            min_cost_of_all = cost
            final_input_of_all = final_input

    return final_input_of_all, best_route_of_all, min_cost_of_all

In [ ]:
def allocatePassengerSeats(adjacency_matrix, filtered_pass_data, thresholds):
# Passengers with higher denails and eralier booking time should be at the top

# Now select top 8 records from the data and start iterations:
# Iter 1: min_cost route is found, now select other 10 people from the list to perform some more permuations/combinations
# If another min_cost route found then include that path as a BUS route and select those 8 passengers for BUS 1

# Remove allocated passangers from waiting queue and increas the denial count against passenegers evluated in Iter 1
# Iter 2: Now, select another 8 passengers from the remaining list and perform MST. Again select 10 more passenegers and look for possible MST
# If MST is found for any passenger(s) then include it as a route for BUS 2 and allocate seats to those 8 passengers

# Repeat above till algorithm finishes with BUS 5 or all records are processed.

# retun dictionary of {BUS:{passenger_ids, route, cost}}, updated_passenger_data_file(basically with updated denial ccounts)
#     sorted_pass_data = sorted(filtered_pass_data, key=lambda x: (-x.denials, x.booking_timestamp))
    buses = []
    # print("filtered_pass_data len before", len(filtered_pass_data))

    for i in range(thresholds["max_buses"]):
        # exit if no passengers remaining
        if not len(filtered_pass_data):
            break

        # passengers = sorted_pass_data[:8]
        # shortest path algo here
        passenger_ids, best_route, min_cost = process_passenger_batch(filtered_pass_data)
        bus = dict({
            "passenger_ids": passenger_ids,
            "route": best_route,
            "cost": min_cost
        })
        buses.append(bus)
        # Pop added passengers from list
        filtered_pass_data = filtered_pass_data[~filtered_pass_data["passenger_id"].isin(passenger_ids)]

        # update denial count after all iterations
        filtered_pass_data["denials"] += 1
        # print("filtered_pass_data len after", len(filtered_pass_data))


    return buses

In [ ]:
# invoke passenger seats allocation algorithm
print(allocatePassengerSeats(matrix, filtered_data, thresholds))